In [1]:
!pip install scikit_posthocs -q
!pip install deepsig -q

In [1]:
import sqlite3
import pandas as pd
from ana_utils import *
import numpy as np
from deepsig import aso
from tqdm.auto import tqdm

c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
METRIC = "r2_score"

# DeepSig
debug = True
seed = 1234
np.random.seed(seed)
min_eps_thr = 0.2
if debug:
    n_sig = 5
else:
    n_sig = 1000

In [10]:
def sec2str(seconds):
    if seconds < 60:
        return str(seconds) + "s"
    elif seconds < 60*60:
        return str(seconds // 60) + "min"
    else:
        return str(seconds // (60*60)) + "h"

conn_ctr = sqlite3.connect(ctr_path)
#df = pd.read_sql(f"SELECT model, search_space_hash, {METRIC}, data_config_hash, data_id, search_space_name FROM results ", conn)
df_ctr = pd.read_sql(f"SELECT results.model as model, search_space_name as Model, {METRIC}, data_config_hash, data_id FROM results join search_spaces on results.search_space_hash == search_spaces.hash_key", conn_ctr)
conn_ctr.close()
df_ctr = convert_listlike_cols(df_ctr)


conn_vimet = sqlite3.connect(vimet_path)
df_vimet = pd.read_sql(f"SELECT results.model as model, search_space_name as Model, {METRIC}, data_config_hash, data_id FROM results join search_spaces on results.search_space_hash == search_spaces.hash_key", conn_vimet)
conn_vimet.close()
df_vimet['Model'] = df_vimet['Model'].str.replace(r'^(.*?)_(.*)$', r'\1(\2)', regex=True)
df_vimet = convert_listlike_cols(df_vimet)

#list(df[df["model"]=="AutoGluon"]["config"])[0]
# df["Model"] = df[["model", "fit_kwargs"]].apply(lambda x: x["model"] if x["fit_kwargs"] is None else f"{x['model']} ({sec2str(json.loads(x['fit_kwargs'])['time_limit'])})", axis=1)
# df = df.drop(["fit_kwargs"], axis=1)
# df[df["model"] == "AutoGluon"]
df_vimet.tail(50)

r2_score
r2_score


,model,Model,r2_score,data_config_hash,data_id,r2_score_corrected
2286,AutoGluon,AutoGluon(10min),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,94,"[0, 0, 0]"
2287,AutoGluon,AutoGluon(10min),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,101,"[0, 0, 0]"
2288,AutoGluon,AutoGluon(10min),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,100,"[0, 0, 0]"
2289,AutoGluon,AutoGluon(10min),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,96,"[0, 0, 0]"
2290,AutoGluon,AutoGluon(10min),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,98,"[0, 0, 0]"
2291,AutoGluon,AutoGluon(10min),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,99,"[0, 0, 0]"
2292,AutoGluon,AutoGluon(10min),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,97,"[0, 0, 0]"
2293,AutoGluon,AutoGluon(1h),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,0,"[0, 0, 0]"
2294,AutoGluon,AutoGluon(1h),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,35,"[0, 0, 0]"
2295,AutoGluon,AutoGluon(1h),"[None, None, None]",48a132abc7a3eea079d82a463bff88687bd88e3ac26e5f...,1,"[0, 0, 0]"


In [12]:
df_vimet['Model'].value_counts()

Model
BayesianRidge              142
ElasticNet                 132
Dummy                      118
XGBoost                    104
LassoLars                  102
MLP                        102
AdaBoost                   101
SGD                        101
Bagging                    101
M5                         100
KNN                        100
RidgeRegressor              99
PassiveAggressive           99
DecisionTree                98
ExtraTrees                  98
Lasso                       98
PartialLeastSquares         97
Huber                       97
GradientBoosting            97
CatBoost                    92
Tweedie                     69
GaussianProcess             61
LARS                        43
AutoGluon(1h)               43
PolynomialRegression        13
RandomForest                11
AutoGluon(10min)             8
AutoGluon(5min)              5
LightGBM                     4
SupportVectorRegression      1
Name: count, dtype: int64

In [13]:
# groups = df_ctr.groupby(by=["data_config_hash", "data_id"])
# group_collection = []
# all_models = df_ctr["Model"].unique()

# baselines = ["CatBoost", "ExtraTrees", "AutoGluon(1h)", "AutoGluon(2h)"]
# beat_baseline = {b: {m: 0 for m in all_models} for b in baselines} # baseline: {model: wins}

# for (hash, data_id), group_df in groups:
#     model_scores = group_df[["Model", METRIC]]
#     model_scores = model_scores.set_index(model_scores["Model"])
#     model_scores_dict = model_scores[METRIC].to_dict()
#     for baseline_model in baselines:
#         if baseline_model not in model_scores_dict:
#             continue
#         for model in model_scores_dict.keys():
#             min_eps = aso(model_scores_dict[model], model_scores_dict[baseline_model], seed=seed)
#             if min_eps < min_eps_thr:
#                 beat_baseline[baseline_model][model] += 1
#     # eps_min = multi_aso(model_scores_dict, confidence_level=0.95, return_df=True, seed=seed) #Too slow and convoluted but interesting
#     break
# beat_baseline

In [14]:
# Name 
# Avg Normalized Performance
# Avg Normalized Performance (Without AG) 
# Avg Rank
# Avg Rank (Without AG) ~ To answer is the weakest AG already the best method
# Avg Absolute Inference Time per sample
# sig_wins = pd.DataFrame(beat_baseline).fillna(0).astype(int)
# sig_wins = sig_wins.rename({col: f"Sig Wins against {col}" for col in sig_wins.columns}, axis=1)
# sig_wins

In [ ]:
def calc_statistics(df):
    groups = df.groupby(by=["data_config_hash", "data_id"])
    group_collection = []
    all_models = df["Model"].unique()
    baselines = ["CatBoost", "ExtraTrees", "AutoGluon(1h)", "AutoGluon(2h)"]
    beat_baseline = {b: {m: 0 for m in all_models} for b in baselines} # baseline: {model: wins}
    
    for (hash, data_id), group_df in tqdm(groups):
        # Calculate Significant Wins against selected baselines
        model_scores = group_df[["Model", METRIC]]
        model_scores = model_scores.set_index(model_scores["Model"])
        model_scores_dict = model_scores[METRIC].to_dict()
        for baseline_model in tqdm(baselines, desc="DeepSig Significance Tests"):
            if baseline_model not in model_scores_dict:
                continue
            for model in model_scores_dict.keys():
                min_eps = aso(model_scores_dict[model], model_scores_dict[baseline_model], seed=seed, show_progress=False, num_bootstrap_iterations=n_sig)
                if min_eps < min_eps_thr:
                    beat_baseline[baseline_model][model] += 1

        
        # Because if worse than zero we dont really care by how much
        # group_df.loc[group_df["avg_metric"] < 0, "avg_metric"] = 0 # Obsolete as moved to outside
        
        # Set Missing Entries to 0 - they likely crahsed
        group_df = group_df.set_index(group_df["Model"])
        group_df = group_df[~group_df.index.duplicated(keep="first")]
        group_df = group_df.drop(["Model"], axis=1)
        group_df = group_df.reindex(all_models, fill_value=0)
        group_df["Model"] = group_df.index
        group_df = group_df.reset_index(drop=True)
        
        maximum = group_df["avg_metric"].max()
        median = group_df["avg_metric"].median()
        if maximum <= 0.001:
            group_df["normalized_metric"] = 0 # If all fail, all fail simple as that
        else:
            group_df["normalized_metric"] = (group_df["avg_metric"] - median) / (maximum-median)
        assert group_df["normalized_metric"].max() < 1.2
        group_df["rank"] = group_df["avg_metric"].rank(method="dense", ascending=False)
        group_df["rounded_rank"] = group_df["avg_metric"].round(2).rank(method="dense", ascending=False)
        group_df["losses(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].max())
        group_df["true losses(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].max()) & (group_df["rounded_rank"] != group_df["rounded_rank"].min())
        group_df["true losses(raw)"] = (group_df["rank"] == group_df["rank"].max()) & (group_df["rank"] != group_df["rank"].min())
        group_df["true wins(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].min()) & (group_df["rounded_rank"] != group_df["rounded_rank"].max())
        group_df["true wins(raw)"] = (group_df["rank"] == group_df["rank"].min()) & (group_df["rank"] != group_df["rank"].max())
    
        group_df_wo_ag = group_df[group_df["model"] != "AutoGluon"]
        maximum_wo_ag = group_df_wo_ag["avg_metric"].max()
        median_wo_ag = group_df_wo_ag["avg_metric"].median()
        if maximum_wo_ag <= 0.001:
            group_df["normalized_metric_wo_ag"] = 0 # If all fail, all fail simple as that
        else:
            group_df["normalized_metric_wo_ag"] = (group_df["avg_metric"] - median_wo_ag) / (maximum_wo_ag-median_wo_ag)
            
        #group_df["rank_wo_ag"] = group_df["avg_metric"].rank(ascending=False)
    
        gavg = (
            group_df.groupby("model", as_index=False)
              .agg(avg_metric=("avg_metric", "max"))
        )
        
        # 2) rank groups by that average
        gavg["rank_wo_ag"] = gavg["avg_metric"].rank(method="dense", ascending=False)
        gavg["rank_wo_ag_round"] = gavg["avg_metric"].round(2).rank(method="dense", ascending=False)
        rank_wo_ag_round = gavg[gavg["model"]!="AutoGluon"]["avg_metric"].round(2).rank(method="dense", ascending=False)
        gavg["victories_wo_ag(ties)"] = (rank_wo_ag_round == 1)
        gavg["victories_wo_ag"] = gavg[gavg["model"]!="AutoGluon"]["avg_metric"].rank(method="dense", ascending=False) == 1
        
        
        # 3) merge back — no restructuring of df
        group_df = group_df.merge(gavg[["model", "rank_wo_ag", "rank_wo_ag_round", "victories_wo_ag", "victories_wo_ag(ties)"]], on="model", how="left")
        #print(group_df[group_df["model"] == "AutoGluon"]["rank_wo_ag"])
        # if list(group_df[group_df["model"] == "AutoGluon"]["rank_wo_ag_round"])[0] > 1:
        #     print(maximum, gavg[gavg["model"] == "AutoGluon"][["rank_wo_ag_round", "avg_metric"]])
        
        autogluon_names = list(group_df[group_df["model"]=="AutoGluon"]["Model"])
        contestants = autogluon_names + ["CatBoost", "ExtraTrees"]
        ag_max_perf = 0
        for contestant in contestants:
            ag_perf = group_df[group_df["Model"] == contestant]["avg_metric"].iloc[0]
            group_df[f"victories_against_{contestant}"] = group_df["avg_metric"] > ag_perf
            group_df[f"major_victories_against_{contestant}"] = group_df["avg_metric"] > (ag_perf*1.01)
            if ag_perf > ag_max_perf:
                ag_max_perf = ag_perf
        group_df[f"victories_against_best_ag"] = group_df["avg_metric"] > ag_max_perf
        group_df[f"major_victories_against_best_ag"] = group_df["avg_metric"] > (ag_max_perf*1.01)
        
    
        group_df["failures"] = (group_df["avg_metric"] <= 0.001) | group_df["avg_metric"].isna()
        group_df = group_df.reset_index(drop=False)
        # TODO Victories against AG (x/y/z/W)
        # TODO Victories without AG
        # best_models = list(group_df[group_df["avg_metric"] >= maximum*0.999]["model"])
        # if not "AutoGluon" in best_models:
            
        #     print(maximum, best_models)
        #     print(group_df[group_df["model"] == "AutoGluon"]["avg_metric"])
        group_collection.append(group_df)
    df_ = pd.concat(group_collection)
    
    df_ = df_.groupby(by=["Model"]).agg(
        Avg_R2 = ("avg_metric", "mean"),
        Rank = ("rank", "mean"),
        Rank_round = ("rounded_rank", "mean"),
        Rank_wo_AG = ("rank_wo_ag", "mean"), # Without counting AG several times
        Rank_wo_AG_round = ("rank_wo_ag_round", "mean"),
        Norm_R2 = ("normalized_metric", "mean"),
        Norm_R2_min=("normalized_metric", "min"),
        Norm_R2_max=("normalized_metric", "max"),
        Norm_R2_wo_AG = ("normalized_metric_wo_ag", "mean"),
        Norm_R2_wo_AG_min = ("normalized_metric_wo_ag", "min"),
        Norm_R2_wo_AG_max = ("normalized_metric_wo_ag", "max"),
        Wins_wo_AG = ("victories_wo_ag", "sum"),
        Wins_Against_AG_5min = ("victories_against_AutoGluon(5min)", "sum"),
        # TODO  Wins_Against_AG_30min = ("victories_against_AutoGluon(30min)", "sum"),
        Wins_Against_AG_1h = ("victories_against_AutoGluon(1h)", "sum"),
        # TODO Wins_Against_AG_2h = ("victories_against_AutoGluon(2h)", "sum"),
        Major_Wins_Against_AG_1h = ("major_victories_against_AutoGluon(1h)", "sum"),
        Wins_Against_AG_best = ("victories_against_best_ag", "sum"),
        Major_Wins_Against_AG_best = ("major_victories_against_best_ag", "sum"),
        Wins_Against_CatBoost = ("victories_against_CatBoost", "sum"),
        Wins_Against_ExtraTrees = ("victories_against_ExtraTrees", "sum"),
        Losses_tied = ("losses(tied)", "sum"),
        True_Losses_tied = ("true losses(tied)", "sum"),
        True_Losses_raw = ("true losses(raw)", "sum"),
        True_Wins_tied = ("true wins(tied)", "sum"),
        True_Wins_raw = ("true wins(raw)", "sum"),
        failures = ("failures", "sum"),
    )

    sig_wins = pd.DataFrame(beat_baseline).fillna(0).astype(int)
    sig_wins = sig_wins.rename({col: f"Sig Wins against {col}" for col in sig_wins.columns}, axis=1)
    df_ = df_.join(sig_wins)
    print(group_df[METRIC])
    return df_.sort_values(by=["True_Wins_tied", "Rank"])

In [19]:
# TODO Create Second Version with uncorrected results for appendix
df_ctr[METRIC] = df_ctr[METRIC+"_corrected"] # Comment out
df_ctr["avg_metric"] = df_ctr[METRIC].apply(
    lambda xs: np.nanmean([x if x is not None else np.nan for x in xs])
)
df_vimet[METRIC] = df_vimet[METRIC+"_corrected"] # Comment out
df_vimet["avg_metric"] = df_vimet[METRIC].apply(
    lambda xs: np.nanmean([x if x is not None else np.nan for x in xs])
)
# df_ctr["avg_metric"] = df_ctr[METRIC+"_corrected"].apply(
#     lambda x: sum(x) / len(x) if x else None
# )
df_ctr_ready = calc_statistics(df_ctr)
df_vimet_ready = calc_statistics(df_vimet)
df_vimet_ready

  0%|          | 0/35 [00:00<?, ?it/s]c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\deepsig\aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")
  3%|▎         | 1/35 [00:10<06:03, 10.68s/it]c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\deepsig\aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")
  6%|▌         | 2/35 [00:20<05:30, 10.02s/it]c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\deepsig\aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")
  9%|▊         | 3/35 [00:29<05:07,  9.60s/it]c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\deepsig\aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in vio

0     [0.5793805469852513, 0.5453118862684447, 0.549...
1     [0.6733920150020394, 0.5979700933376837, 0.680...
2                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3     [0.5792544481289225, 0.5453003021882409, 0.547...
4     [0.5784533260898252, 0.47610974362792224, 0.52...
5     [0.27275993231209217, 0.4031192539418823, 0.43...
6                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7     [0.6630774365524876, 0.6114265505760741, 0.695...
8     [0.7210061238885122, 0.6421096416935099, 0.686...
9     [0.6926871390811642, 0.6448541529622249, 0.658...
10    [0.5461906343060163, 0.5287785497113879, 0.511...
11    [0.5797947412481897, 0.5457668091022094, 0.544...
12    [0.5784334053940771, 0.5453299970526431, 0.549...
13    [0.5781572107491826, 0.5453795931097871, 0.554...
14    [0.717711922235138, 0.6715922253658309, 0.7078...
15    [0.5767595560866273, 0.5374384668351385, 0.515...
16    [0.6895286222413359, 0.6671412276007342, 0.702...
17                       [0, 0, 0, 0, 0, 0, 0, 0

  0%|          | 0/102 [00:00<?, ?it/s]c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\deepsig\aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")
  1%|          | 1/102 [00:05<09:07,  5.42s/it]c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\deepsig\aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")
  2%|▏         | 2/102 [00:11<09:39,  5.80s/it]c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\deepsig\aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")
  3%|▎         | 3/102 [00:15<08:11,  4.97s/it]c:\Users\poehlmann\PycharmProjects\AutoViMet\.venv\Lib\site-packages\deepsig\aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in

0                        [0.3619168593784522, 0, 0]
1                       [0.45454783702866286, 0, 0]
2                        [0.1417432950026002, 0, 0]
3                                         [0, 0, 0]
4                       [0, 0, 0.10606331968939697]
5                                         [0, 0, 0]
6                                         [0, 0, 0]
7                       [0.23905003898916233, 0, 0]
8     [0.14210403086580092, 0, 0.11910311159105225]
9                       [0.31848211571810336, 0, 0]
10                      [0.11863015775179542, 0, 0]
11                                        [0, 0, 0]
12                      [0.36429391713311354, 0, 0]
13                      [0.04926649896531199, 0, 0]
14                      [0.05274393378482656, 0, 0]
15                                        [0, 0, 0]
16                                                0
17                      [0.31839762196500143, 0, 0]
18                                        [0, 0, 0]
19          

,Avg_R2,Rank,Rank_round,Rank_wo_AG,Rank_wo_AG_round,Norm_R2,Norm_R2_min,Norm_R2_max,Norm_R2_wo_AG,Norm_R2_wo_AG_min,...,Losses_tied,True_Losses_tied,True_Losses_raw,True_Wins_tied,True_Wins_raw,failures,Sig Wins against CatBoost,Sig Wins against ExtraTrees,Sig Wins against AutoGluon(1h),Sig Wins against AutoGluon(2h)
Model,,,,,,,,,,,,,,,,,,,,,
PolynomialRegression,0.012155,11.450980,8.970588,11.450980,8.970588,-inf,-inf,0.859655,-inf,-inf,...,98,96,96,0,0,98,0,0,3,0
PassiveAggressive,0.003147,11.529412,9.068627,11.529412,9.068627,-inf,-inf,0.540082,-inf,-inf,...,97,95,92,0,0,95,0,1,4,0
MLP,0.000112,11.666667,9.156863,11.666667,9.156863,-inf,-inf,0.058920,-inf,-inf,...,101,99,99,0,0,101,1,0,0,0
AutoGluon(10min),0.000000,11.686275,9.166667,11.686275,9.166667,-inf,-inf,0.000000,-inf,-inf,...,102,100,100,0,0,102,0,0,0,0
AutoGluon(1h),0.000000,11.686275,9.166667,11.686275,9.166667,-inf,-inf,0.000000,-inf,-inf,...,102,100,100,0,0,102,0,0,0,0
AutoGluon(5min),0.000000,11.686275,9.166667,11.686275,9.166667,-inf,-inf,0.000000,-inf,-inf,...,102,100,100,0,0,102,0,0,0,0
SGD,0.000000,11.686275,9.166667,11.686275,9.166667,-inf,-inf,0.000000,-inf,-inf,...,102,100,100,0,0,102,0,0,0,0
ElasticNet,0.122176,9.039216,7.137255,9.039216,7.137255,0.179030,-0.418133,0.987264,0.176220,-0.418133,...,50,48,47,1,1,49,1,1,21,0
BayesianRidge,0.062713,10.235294,8.107843,10.235294,8.107843,0.079254,-0.241987,0.910131,0.072690,-0.547663,...,75,73,73,1,1,75,2,4,11,0


In [23]:
def bold_if(value, best_value):
    if isinstance(value, int):
        if value == best_value:
            return f"\\textbf{{{value}}}"
        else:
            return f"{value}"
    else: 
        if value == best_value:
            return f"\\textbf{{{value:.2f}}}"
        else:
            return f"{value:.2f}"


def prepare_table(df):
    def pretty(x, value_name):
        return bold_if(x[value_name], best_entries[value_name])
    df_ctr_ready = df.copy()
    best_entries = {
        "Norm_R2_min": df_ctr_ready["Norm_R2_min"].max(),
        "Norm_R2_max": df_ctr_ready["Norm_R2_max"].max(),
        "Norm_R2": df_ctr_ready["Norm_R2"].max(),
        "Norm_R2_wo_AG_min": df_ctr_ready["Norm_R2_wo_AG_min"].max(),
        "Norm_R2_wo_AG_max": df_ctr_ready["Norm_R2_wo_AG_max"].max(),
        "Norm_R2_wo_AG": df_ctr_ready["Norm_R2_wo_AG"].max(),
        "Rank": df_ctr_ready["Rank"].min(),
        "Rank_round": df_ctr_ready["Rank_round"].min(),
        "True_Losses_tied": df_ctr_ready["True_Losses_tied"].min(),
        "True_Losses_raw": df_ctr_ready["True_Losses_raw"].min(),
        "True_Wins_tied": df_ctr_ready["True_Wins_tied"].max(),
        "True_Wins_raw": df_ctr_ready["True_Wins_raw"].max(),
        "failures": df_ctr_ready["failures"].min(),
        "Wins_Against_AG_1h": df_ctr_ready["Wins_Against_AG_1h"].max(),
        # TODO "Wins_Against_AG_2h": df_ctr_ready["Wins_Against_AG_2h"].max(),
        "Wins_Against_CatBoost": df_ctr_ready["Wins_Against_CatBoost"].max(),
        "Wins_Against_ExtraTrees": df_ctr_ready["Wins_Against_ExtraTrees"].max(),
        "Sig Wins against AutoGluon(1h)": df_ctr_ready["Sig Wins against AutoGluon(1h)"].max(),
        # TODO "Sig Wins against AutoGluon(2h)": df_ctr_ready["Sig Wins against AutoGluon(2h)"].max(),
        "Sig Wins against CatBoost": df_ctr_ready["Sig Wins against CatBoost"].max(),
        "Sig Wins against ExtraTrees": df_ctr_ready["Sig Wins against ExtraTrees"].max(),
    }
    df_ctr_ready["Norm R2"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Norm_R2_min')}/{pretty(x, 'Norm_R2')}/{pretty(x, 'Norm_R2_max')}", axis=1)
    df_ctr_ready["Norm R2 (wo AG)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Norm_R2_wo_AG_min')}/{pretty(x, 'Norm_R2_wo_AG')}/{pretty(x, 'Norm_R2_wo_AG_max')}", axis=1)
    df_ctr_ready["Avg Rank"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Rank')} ({pretty(x, 'Rank_round')})", axis=1)
    df_ctr_ready["Wins"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'True_Wins_tied')} ({pretty(x, 'True_Wins_raw')})", axis=1) # Based on rounded Rank + Ignore Wins when EVERY model performs the same
    df_ctr_ready["Losses"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'True_Losses_tied')} ({pretty(x, 'True_Losses_raw')})", axis=1) # Based on rounded Rank + Ignore Losses when EVERY model performs terrible
    df_ctr_ready["Failures"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'failures')}", axis=1) 

    df_ctr_ready["Wins (AG 1h)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Wins_Against_AG_1h')} ({pretty(x, 'Sig Wins against AutoGluon(1h)')})", axis=1) 
    # TODO df_ctr_ready["Wins (AG 2h)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Wins_Against_AG_2h')} ({pretty(x, 'Sig Wins against AutoGluon(2h)')})", axis=1) 
    df_ctr_ready["Wins (CatBoost)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Wins_Against_CatBoost')} ({pretty(x, 'Sig Wins against CatBoost')})", axis=1) 
    df_ctr_ready["Wins (ExtraTrees)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Wins_Against_ExtraTrees')} ({pretty(x, 'Sig Wins against ExtraTrees')})", axis=1) 

    df_for_render = df_ctr_ready[["Norm R2", "Norm R2 (wo AG)", "Avg Rank", "Wins", "Losses", "Failures", "Wins (AG 1h)", "Wins (CatBoost)", "Wins (ExtraTrees)"]]
    return df_for_render

df_ctr_table = prepare_table(df_ctr_ready)
df_vimet_table = prepare_table(df_vimet_ready)
df_ctr_table

,Norm R2,Norm R2 (wo AG),Avg Rank,Wins,Losses,Failures,Wins (AG 1h),Wins (CatBoost),Wins (ExtraTrees)
Model,,,,,,,,,
GaussianProcess,-3.52/-0.20/0.88,-2.81/0.17/1.00,18.31 (7.40),0 (0),1 (1),1,0 (0),6 (2),8 (6)
Bagging,-10.78/-0.73/0.88,-5.26/-0.19/0.96,20.23 (8.43),0 (0),\textbf{0} (\textbf{0}),\textbf{0},1 (0),2 (0),2 (2)
KNN,-18.79/-1.24/0.70,-14.94/-0.57/0.86,20.37 (8.60),0 (0),1 (1),1,0 (0),3 (0),3 (2)
MLP,-15.82/-1.78/0.95,-11.44/-0.82/1.00,22.11 (9.71),0 (0),5 (2),3,1 (1),4 (2),4 (1)
Tweedie,-51.73/-2.47/0.60,-27.02/-1.03/0.92,22.29 (9.26),0 (0),5 (5),5,0 (0),2 (1),1 (1)
SupportVectorRegression,-51.73/-4.87/0.87,-27.02/-2.41/0.98,24.43 (10.91),0 (0),17 (16),16,0 (0),4 (3),2 (2)
CubistModel,-69.74/-10.28/-0.03,-33.05/-5.71/-0.06,30.66 (14.51),0 (0),35 (35),35,0 (0),0 (0),0 (0)
Dummy,-69.74/-10.28/-0.03,-33.05/-5.71/-0.06,30.66 (14.51),0 (0),35 (35),35,0 (0),0 (0),0 (0)
ElasticNet,-3.19/-0.55/1.00,-0.58/0.07/1.00,19.86 (8.37),1 (0),1 (\textbf{0}),\textbf{0},3 (2),6 (5),3 (3)


In [24]:
combined = pd.concat(
    {"ctr": df_ctr_table, "vimet": df_vimet_table}, # TODO Add ViMet
    axis=1
)

# Optional: put metric first, source second -> col -> (ctr/vimet)
combined = combined.swaplevel(0, 1, axis=1).sort_index(axis=1)
combined

Avg Rank  \
                                                   ctr   
Model                                                    
GaussianProcess                           18.31 (7.40)   
Bagging                                   20.23 (8.43)   
KNN                                       20.37 (8.60)   
MLP                                       22.11 (9.71)   
Tweedie                                   22.29 (9.26)   
SupportVectorRegression                  24.43 (10.91)   
CubistModel                              30.66 (14.51)   
Dummy                                    30.66 (14.51)   
ElasticNet                                19.86 (8.37)   
LassoLars                                 20.09 (8.60)   
LARS                                      22.37 (9.57)   
Huber                                     22.91 (9.71)   
SGD                                      27.69 (12.03)   
PassiveAggressive                        28.74 (13.31)   
BayesianRidge                             18.43 (8.09)   
RidgeRegressor                            18.74 (8.20)   
DecisionTree                              20.17 (8.20)   
Lasso                                     20.20 (8.51)   
PartialLeastSquares                       20.77 (8.51)   
RandomForest                              18.11 (7.71)   
PolynomialRegression                      20.06 (8.86)   
XGBoost                                   15.26 (6.14)   
LightGBM                                  11.03 (3.89)   
GradientBoosting                          12.03 (4.23)   
AdaBoost                                  13.20 (4.69)   
M5                                        12.09 (4.14)   
AutoGluon(1min)                           10.00 (3.20)   
ExtraTrees                                10.77 (3.77)   
CatBoost                                  10.17 (3.46)   
AutoGluon(5min)                            6.14 (2.14)   
AutoGluon(30min)                           4.34 (1.69)   
AutoGluon(10min)                           4.89 (1.74)   
AutoGluon(2h)                              3.97 (1.77)   
AutoGluon(1h)            \textbf{3.43} (\textbf{1.34})   

                                                          Failures  \
                                                 vimet         ctr   
Model                                                                
GaussianProcess                           10.28 (8.11)           1   
Bagging                                    6.47 (5.03)  \textbf{0}   
KNN                                        9.09 (7.23)           1   
MLP                                       11.67 (9.16)           3   
Tweedie                                   11.02 (8.66)           5   
SupportVectorRegression                   11.68 (9.16)          16   
CubistModel                                        NaN          35   
Dummy                                     11.68 (9.16)          35   
ElasticNet                                 9.04 (7.14)  \textbf{0}   
LassoLars                                  8.81 (6.97)           1   
LARS                                      11.00 (8.66)           4   
Huber                                     10.93 (8.60)           2   
SGD                                       11.69 (9.17)           7   
PassiveAggressive                         11.53 (9.07)           7   
BayesianRidge                             10.24 (8.11)           1   
RidgeRegressor                            10.32 (8.24)  \textbf{0}   
DecisionTree                               7.30 (5.73)           1   
Lasso                                      9.46 (7.47)  \textbf{0}   
PartialLeastSquares                        9.25 (7.32)  \textbf{0}   
RandomForest                              10.65 (8.45)          12   
PolynomialRegression                      11.45 (8.97)           8   
XGBoost                                    5.09 (3.92)           7   
LightGBM                                  11.64 (9.12)           1   
GradientBoosting                           4.46 (3.58)           1   
AdaBoost 

In [10]:
df_["Model"] = df_.index

df_to_latex_table(
    df=df_[["Model", "normalized_metric_wo_ag", "normalized_metric", "rounded_rank", "true losses(tied)", "failures"]],
    filepath="./cache/ranking.tex",
    caption="Performance comparison of all evaluated methods against AutoGluon",
    label="tab:ranking",
    float_fmt=".2f",
)

'\\begin{table}[t]\n\\centering\n\\begin{tabular}{lrrrrr}\n\\toprule\n Model                   &   normalized\\_metric\\_wo\\_ag &   normalized\\_metric &   rounded\\_rank &   true losses(tied) &   failures \\\\\n\\midrule\n AdaBoost                &                      0.36 &                0.17 &           5.4  &                   3 &          4 \\\\\n AutoGluon (10min)       &                      0.92 &                0.82 &           1.66 &                   0 &          1 \\\\\n AutoGluon (1h)          &                      1    &                0.91 &           1.51 &                   0 &          1 \\\\\n AutoGluon (1min)        &                      0.21 &                0.07 &           2.26 &                   0 &          1 \\\\\n AutoGluon (30min)       &                      1    &                0.91 &           1.49 &                   0 &          1 \\\\\n AutoGluon (4h)          &                      0.95 &                0.85 &           1.66 &                  

In [8]:
# TODO 
# Get Names of outlier DS describe what happened (fair but minor, fair server but only two models better, all shit)

In [13]:
import scikit_posthocs as sp
import matplotlib.pyplot as plt

fig = sp.critical_difference_diagram(
    df.values,
    df.columns.tolist(),
    alpha=0.05,  # Nemenyi
)
plt.show()


TypeError: critical_difference_diagram() got an unexpected keyword argument 'alpha'